In [17]:
%%capture --no-stderr
%pip install langchainhub langchain langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community

In [12]:
import getpass
import os


from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')



In [18]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

목적별로 이런식으로 prompt를 작성하면 좋겠다
이런 것들을 모아둔것

In [20]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is the process of breaking down complex tasks into smaller, manageable steps to enhance understanding and execution. It often involves techniques like Chain of Thought (CoT) prompting, where models think step-by-step, and Tree of Thoughts, which explores multiple reasoning paths at each step. This approach can be facilitated by simple prompting, task-specific instructions, or human input.'

In [23]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.0 MB/s eta 0:00:00


In [24]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("/content/drive/MyDrive/Untitled Folder/state_of_the_union.txt")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)

retriever = vectorstore.as_retriever()

docs = retriever.invoke("what did the president say about ketanji brown jackson?")


In [25]:
# Retrieve more documents with higher diversity
# Useful if your dataset has many similar documents
vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.25}
)

# Fetch more documents for the MMR algorithm to consider
# But only return the top 5
vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'fetch_k': 50}
)

# Only retrieve documents that have a relevance score
# Above a certain threshold
vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.8}
)

# Only get the single most similar document from the dataset
vectorstore.as_retriever(search_kwargs={'k': 1})

# Use a filter to only retrieve documents from a specific paper
vectorstore.as_retriever(
    search_kwargs={'filter': {'paper_title':'GPT-4 Technical Report'}}
)

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x792227802050>, search_kwargs={'filter': {'paper_title': 'GPT-4 Technical Report'}})

In [31]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

joke_query = "Tell me a joke."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": joke_query})

{'joke': "Why don't scientists trust atoms? Because they make up everything!"}

In [33]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

```json
{
  "definition": "Task decomposition is the process of breaking down a complex task or problem into smaller, more manageable sub-tasks or components. This approach helps in simplifying the overall task, making it easier to understand, execute, and manage.",
  "benefits": [
    "Enhances clarity and understanding of tasks",
    "Facilitates better planning and resource allocation",
    "Allows for parallel execution of sub-tasks",
    "Improves problem-solving by isolating specific issues",
    "Helps in tracking progress and identifying bottlenecks"
  ],
  "applications": [
    "Project management",
    "Software development",
    "Research and analysis",
    "Education and learning strategies",
    "Personal productivity and goal setting"
  ]
}
```

In [37]:
parser = JsonOutputParser()

prompt = PromptTemplate(
    template="""
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are an AI language model assistant. check the relevance between documents given and query<|eot_id|>\n
            <|start_header_id|>user<|end_header_id|>\n{format_instructions}\n{query}\n<|eot_id|>\n
            <|start_header_id|>documents<|end_header_id|>{documents}<|eot_id|>\n
            <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["query","documents","format_instructions"],
    partial_variables={"format_instructions": parser.get_format_instructions()+"""For example, if there is relevance then just return {"relevance": "yes"} otherwise return {"relevance": "no"}. just a json object not any description"""},
)

chain = {"documents": retriever | format_docs, "query": RunnablePassthrough()} | prompt | llm | parser #StrOutputParser()

print(chain.invoke("What is Task Decomposition?"))

print(chain.invoke("With a duty to one another to the American people to the Constitution"))

print(chain.invoke("Did Putin meet with Ukrainian people?"))

{'relevance': 'no'}
{'relevance': 'yes'}
{'relevance': 'yes'}


In [47]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


prompt = PromptTemplate(
    template="""
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are an AI language model assistant. check the relevance between documents given and query<|eot_id|>\n
            <|start_header_id|>user<|end_header_id|>\n{format_instructions}\n{query}\n<|eot_id|>\n
            <|start_header_id|>documents<|end_header_id|>{documents}<|eot_id|>\n
            <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["query","documents","format_instructions"],
    partial_variables={"format_instructions": parser.get_format_instructions()+"""For example, if there is hallucination then just return {"hallucination": "yes"} otherwise return {"hallucination": "no"}. just a json object not any description"""},
)

rag_chain = (
    {"documents": retriever | format_docs, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("putin is russian"):
    print(chunk, end="", flush=True)


{"hallucination":"no"}